In [1]:
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

In [3]:
mcp_config = {
    "math": {
        "command": "python",
        "args": ["/Volumes/Projects/TemporalProjects/McpDemo/a_mcp/math_server.py"],
        "transport": "stdio",
    },
    "weather": {
        "url": "http://localhost:8000/sse",
        "transport": "sse",
    },
}

In [4]:
async def init_agent():
    client = MultiServerMCPClient(mcp_config)
    await client.__aenter__()  # inicia el contexto manualmente

    tools = client.get_tools()
    agent = create_react_agent("openai:gpt-4o", tools)

    return agent, client

In [7]:
async def test_agent(agent):
    # Pregunta matemática
    math_response = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "what's (3 + 5) x 12?"}]}
    )
    print("Math response:", math_response)

    # Pregunta sobre el clima
    weather_response = await agent.ainvoke(
        {"messages": [{"role": "user", "content": "what is the weather in nyc?"}]}
    )
    print("Weather response:", weather_response)

In [8]:
agent, client = await init_agent()
await test_agent(agent)


Math response: {'messages': [HumanMessage(content="what's (3 + 5) x 12?", additional_kwargs={}, response_metadata={}, id='e94efd43-9cef-40fb-9cdd-4da53270874c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ywxwUQamQYNMoW9Z2sP0eySI', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_3Ytgg3o6pPCpuRXm1jY2GSjY', 'function': {'arguments': '{"a": 8, "b": 12}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 96, 'total_tokens': 147, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BUu5rVt37khseNQJBZpQlp1mIUFfJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='

In [9]:
async def ask_agent(agent, question: str):
    response = await agent.ainvoke(
        {"messages": [{"role": "user", "content": question}]}
    )
    print(f"💬 Pregunta: {question}\n🧠 Respuesta: {response}")

In [10]:
await ask_agent(agent, "what's 7 * 6 + 2?")
await ask_agent(agent, "what's the weather like in Paris?")
await ask_agent(agent, "can you tell me the capital of Peru?")

💬 Pregunta: what's 7 * 6 + 2?
🧠 Respuesta: {'messages': [HumanMessage(content="what's 7 * 6 + 2?", additional_kwargs={}, response_metadata={}, id='6c7ad255-bd68-43f6-959c-2dd83c07d5d6'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iAUEgl8Y9B1omT1Kom3sdCs6', 'function': {'arguments': '{"a": 7, "b": 6}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_4SvjvXgrglpKv9gpd8Xoevf3', 'function': {'arguments': '{"a": 42, "b": 2}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 95, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f5bdcc3276', 'id': 'chatcmpl-BUu6FIC1BCmzfS9nyLNCh3yyJpXmr', 'service_tier': 'default', 'finish_reason': 'tool_calls'

In [ ]:
# IMPORTANTE: libera recursos
# await client.__aexit__(None, None, None)